In [1]:
from Data_reading.reading import give_modified_bus_stops_df
from Data_reading.reading import give_modified_lines_stops_df
from Data_reading.reading import give_modified_curr_positions_df
from Data_reading.reading import give_modified_time_tables_df

bus_stops_df = give_modified_bus_stops_df()
lines_stops_df = give_modified_lines_stops_df()
positions_df = give_modified_curr_positions_df()
time_tables_df = give_modified_time_tables_df()

from Data_reading.modifying_dfs import Aliases as als
import datetime

import pandas as pd

INF = 100000

def dict_of_dfs_by_column(dataframe, column_name):
    grouped = dataframe.groupby(column_name)

    dataframes = {}
    for category, group_df in grouped:
        dataframes[category] = group_df 
        
    return dataframes

def give_minutes_between_two_timestamps(t1, t2):
    diff = t2 - t1
    return diff.total_seconds() / 60

def give_general_time_interval(bus_positoins_df):
    min_time = min(bus_positoins_df[als.TIME.value])
    max_time = max(bus_positoins_df[als.TIME.value])
    return min_time, max_time

def calculate_expected_waiting_time(general_time_interval, line_arrives_time):  
    start = general_time_interval[0]
    end = general_time_interval[1]
    times = [start] + line_arrives_time + [end]
    integral = 0
    for i in range(len(times) - 1):
        diff = give_minutes_between_two_timestamps(times[i], times[i+1])
        print(diff)
        integral += diff * diff / 2
        
    interval_length = give_minutes_between_two_timestamps(start, end)
    expected_delay = integral / interval_length
    return expected_delay
        

def calc_delays(list_of_arrival_times, time_table_for_line, time_interval):
    end = time_interval[1]
    delay_sum = 0
    for scheduled_time in time_table_for_line:
        first_time_after = give_minutes_between_two_timestamps(scheduled_time, end)
        
        for real_arrive_times in list_of_arrival_times:
            single_delay = give_minutes_between_two_timestamps(scheduled_time, real_arrive_times)
            if single_delay >= -1:
                first_time_after = min(first_time_after, single_delay, 0)
            
        delay_sum += first_time_after
        
    return delay_sum
        
def dict_with_time_only(dict_of_arrivals_by_line):
    new_dict = {}
    for line, arrivals_time_df in dict_of_arrivals_by_line.items():
        time_list = arrivals_time_df[als.TIME.value].tolist()
        new_dict[line] = time_list
        
    return new_dict
    

# this function is supposed to give dataframe in format:
# (bus_stop_id, expected_waiting)
# expected waiting is for given bus stop is defined this way:
# we randomly select line and then go to given bus stop 
# time we wait is first vehicle of selected line which will appear after we arrive 
# how we are going to calculate it for the given bus stop?
# we will take average of expected time for every line and then for line:
# for line we will take general time interval, and then we will sum up 
# probability of appearing in given minute on bus stop (for every minute the same)
# and amount of time we are going to wait (time to neaarest vehicle of this line)
# so it is going to be eaasy formula:

def give_df_with_expected_waiting_time(arrivals_df, bus_positions_df):
    dict_of_arrivals = dict_of_dfs_by_column(arrivals_df, als.BUS_STOP_ID.value)

    cols = [als.BUS_STOP_ID.value, als.EXPECTED_WAITING.value, als.LAT, als.LON]
    df_data = []

    time_interval = give_general_time_interval(bus_positions_df)
    for bus_stop_id, arrive_df in dict_of_arrivals.items():
        dict_of_arrivals_by_line = dict_of_dfs_by_column(arrive_df, als.LINE.value)
        dict_time_only = dict_with_time_only(dict_of_arrivals_by_line)
        
        sum_of_expected = 0
        for line, list_of_arrival_times in dict_time_only.items():
            sum_of_expected += calculate_expected_waiting_time(time_interval,
                                                               list_of_arrival_times)
            
        num_of_lines = len(dict_time_only)
        final_expected = sum_of_expected / num_of_lines
        
        coords = (arrive_df[als.LAT.value].iloc[0], 
                  arrive_df[als.LON.value].iloc[0])
        df_data.append([bus_stop_id, final_expected, coords[0], coords[1]])
        
    df = pd.DataFrame(df_data, columns = cols)      
    return df

# this function is supposed to give dataframe in format:
# (bus_stop_id, average_delay, bus_stop_lat, bus_stop_lon)
# formula will be average of all delays:
# for given bus stop we want to have the following:
# for every time table time of arrival of line we want to have first arrival after it (or minute earlier)
# then difference between these two times is delay, we take average of them

def get_delays_data(dict_of_arrivals, dict_of_time_tables):
    for bus_stop_id, arrive_df in dict_of_arrivals.items():
        if bus_stop_id not in dict_of_time_tables.keys(): # to erase
            continue 
        else:
            time_table_on_this_bus_stop_df = dict_of_time_tables[bus_stop_id]
        
        dict_time_table_line = dict_of_dfs_by_column(time_table_on_this_bus_stop_df,
                                                     als.LINE.value)
        
        dict_of_arrivals_by_line = dict_of_dfs_by_column(arrive_df, als.LINE.value)
        dict_time_only = dict_with_time_only(dict_of_arrivals_by_line)
        
        sum_delays = 0
        counter = 0
        result_df_data = []
        for line, list_of_arrival_times in dict_time_only.items():
            if line not in dict_time_table_line.keys():
                continue
            time_table_for_line = dict_time_table_line[line]
            sum_delays = calc_delays(list_of_arrival_times, time_table_for_line)
            
            counter += len(list_of_arrival_times)
            sum_delays += count_and_sum_delay[1]
         
        if counter != 0:
            avg_delay = sum_delays / counter 
        else:
            avg_delay = 0
                
        coords = (arrive_df[als.LAT.value].iloc[0], 
                  arrive_df[als.LON.value].iloc[0])
        result_df_data.append([bus_stop_id, avg_delay, coords[0], coords[1]])

    return result_df_data

def give_df_with_avg_delay_on_bus_stop(arrivals_df, time_tables_df):
    dict_of_arrivals = dict_of_dfs_by_column(arrivals_df, als.BUS_STOP_ID.value)
    dict_of_time_tables = dict_of_dfs_by_column(time_tables_df, als.BUS_STOP_ID.value)
    
    cols = [als.BUS_STOP_ID.value, als.DELAY.value, als.LAT, als.LON]
    result_df_data = get_delays_data(dict_of_arrivals, dict_of_time_tables)
        
    df = pd.DataFrame(result_df_data, columns = cols)      
    return df
        
    
    

c:\Users\Admin\Desktop\rzeczy\studia\semestr_3\kurs_python\final_python_project\final_python_project\Data_analysis\Data_reading\modifying_dfs.py:57: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[Aliases.TIME.value] = pd.to_datetime(df[Aliases.TIME.value])


In [2]:
from arrivals import give_df_with_arrivals
arrivals_df = give_df_with_arrivals(positions_df, lines_stops_df, bus_stops_df)

In [3]:
delay_df = give_df_with_expected_waiting_time(arrivals_df, positions_df)
print(delay_df)

423.1166666666667
0.05
422.9166666666667
0.25
423.1
0.06666666666666667
422.8666666666667
0.3
423.05
0.11666666666666667
423.0833333333333
0.08333333333333333
423.1
0.06666666666666667
422.96666666666664
0.2
423.06666666666666
0.1
422.85
0.31666666666666665
422.6
0.5666666666666667
423.1
0.06666666666666667
422.9166666666667
0.25
422.81666666666666
0.35
423.03333333333336
0.13333333333333333
423.1
0.06666666666666667
422.8333333333333
0.3333333333333333
422.8666666666667
0.3
423.06666666666666
0.1
423.06666666666666
0.1
422.9166666666667
0.25
423.06666666666666
-0.2
0.3
423.0833333333333
0.08333333333333333
423.1
0.06666666666666667
422.95
0.21666666666666667
422.8666666666667
0.3
422.8833333333333
0.2833333333333333
423.0833333333333
0.08333333333333333
423.1166666666667
0.05
423.05
0.11666666666666667
422.8833333333333
0.2833333333333333
422.9
0.26666666666666666
422.76666666666665
0.4
423.0833333333333
0.08333333333333333
423.03333333333336
0.13333333333333333
422.6
0.56666666666666